# Loading a Saved Model

This notebook will show you how to load a model which you have trained using a seperate tool such as [Lobe](https://www.lobe.ai/) or [Teachable Machine](https://teachablemachine.withgoogle.com/).

First you will need to train a model using one of the tools above of course. once trained download and save your model - __Make sure you save it in a format suitable to Python! This might not be obvious at first, but carefully look at the descriptions and sample code they give you, you will soon learn to recognise the Python syntax.__

It will likely get downloaded in a `.zip` file. Extract the file and then upload it to __Google Drive__. Now we are ready to get going!

## Connect to Google Drive

This will link you Google Drive to this Colab session so that we can save files there. Run the cell and then you will be given a link to click on in the output. It's just Google Drive asking for permission, and then it will give you a long code which you paste back in a box in the cell output, then press enter.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Easy as that!** If you open the file explorer on the left (click on the little folder icon 📁) you should see the following file structure:

```
.
├── drive
├── MyDrive
│   └── < All the stuff in your Google Drive >
└── sample_data
```

Make sure you can find your model in there... Now we can load it!


## Load the Model

You need the **path** to your model which is just where it exists in your Google Drive folder. The _easiest_ and most _error free_ way of doing this is to find the `.h5` in the file explorer on the left, **right click on it** and select **Copy Path**.

Now you can replace the value of `model_path` below to be what you just copied.

The model path must be a **string** to make sure it is still inside quotation marks. It does not matter whether they are 'single quotes' or "double quotes".

In [10]:
import tensorflow as tf
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# **Change this to the path of your model**
model_path = '/content/drive/MyDrive/dip-ai-comp-futures/model/keras_model.h5'

# Load the model
model = load_model(model_path, compile=False)

## Use the Model

First lets get some data to test our model with. The cell below will allow you to use your webcam to take a photo and save it to this Colab Session.

> This is more for future reference but... **beware** that files saved to the _session_ are not permanent and will be lost as soon as you close this tab. If you want to save a file _forever_ make sure to move it into your Google Drive folder!

In [104]:
# From: https://ricardodeazambuja.com/deep_learning/2019/03/09/audio_and_video_google_colab/

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
from PIL import Image

VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)
  
var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""
def take_photo(filename='photo.jpg', quality=0.8, size=(800,600)):
  display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  f = io.BytesIO(binary)
  #return np.asarray(Image.open(f))
  with open(filename, 'wb') as f:
    f.write(binary)
  return len(binary)

Run the cell below and *click on the video* to save a photo called `photo.jpg`.

In [105]:
img = take_photo() # click

Finally we get to run the model. We will use the photo we just took and run it through the model..

In [109]:
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Replace this with the path to your image
image = Image.open('photo.jpg')
#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)
# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(f"The models prediction is... {prediction}")

The models prediction is... [[0.9972223  0.00277766]]


The model gave us a prediction... but it's just two numbers? The number represent the _probability_ of the image representing each of the classes in your model. In my case there are two classes, and I need to assign that probability to an actual class label:

In [113]:
classes = ['Person', 'Mug']

choice = np.argmax(prediction) # Get the highest predicted value
output = classes[choice]

print(output)

Person


# Nice.